Importação de bibliotecas

In [101]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.decomposition import PCA
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import IsolationForest, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from itertools import combinations
import joblib
from sklearn.inspection import permutation_importance

### 1. Entendimento do dataframe pré-processado

#### 1.1. Lendo o dataframe.

In [102]:
df = pd.read_csv("../data/dataframe.csv")

#### 1.2. Entendendo o dataFrame

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

### 2. Modificação do antigo dataframe

#### 2.1. Adição de colunas extras conforme a demanda dos modelos

&nbsp;&nbsp;&nbsp;&nbsp;Adiciona coluna de gastos tendo em vista o feedback do cliente, pois, o meterIndex é um fator cumulativo, ou seja, sempre aumenta.

In [106]:
df['gain'] = df['gain'].fillna(1)
df['medidor'] = df['pulseCount'] * df['gain']

In [ ]:
df['gasto'] = 0.0

df = df.sort_values(by=['datetime', 'clientCode' ])

df['gasto'] = df.groupby(['clientCode', 'clientIndex'])['medidor'].diff()

df['gasto'] = df['gasto'].fillna(0)

df = df.sort_values(by=[ 'clientCode', 'clientIndex','datetime'])

df.head(10)

Verifica se há números negativos na coluna gastos,
já que a coluna gastos pelo número seguinte sempre ser maior que o anterior não deve ter números negativos:

In [ ]:
gastos_negativos = df[df['gasto'] < 0]
print(gastos_negativos)

#### 2.5. Deletando as colunas que não são mais necessárias.

In [109]:
df = df.drop(columns=['meterIndex', 'inputType', 'rssi', 'Cocção + Aquecedor', 'Cocção + Caldeira', 'Aquecedor', 'Cocção', 'Caldeira', 'Cocção + Aquecedor + Piscina', 'condCode', 'Infinity V2', 'pulseCount', 'gain'])

In [ ]:

df['datetime'] = pd.to_datetime(df['datetime'], unit='s')

df = df[df['gasto'] != 0]

df = df.sort_values(by=['clientCode', 'clientIndex', 'datetime'])

df['diferenca_tempo'] = df.groupby(['clientCode', 'clientIndex'])['datetime'].diff().dt.total_seconds().fillna(3600) / 3600

df['gasto_por_hora'] = round(df['gasto'] / df['diferenca_tempo'], 10)

df = df.drop(columns=['diferenca_tempo'])

df

In [111]:
df['date'] = pd.to_datetime(df['datetime'], unit='s').dt.strftime('%Y-%m-%d')

In [ ]:
df = df.drop(['datetime', 'categoria', 'medidor', 'gasto', 'IG1K-L-v2', 'bairro'], axis=1)

df

### 3. Retirada de pontos outiliers, que podem influenciar negativamente no modelo e separação do DataFrame de treinamento e de teste.
O DataFrame de treinamento corresponde aos meses de fevereiro até maio, enquanto o DataFrame de teste corresponde ao mês de junho.

In [113]:
df['date'] = pd.to_datetime(df['date'])

percentil_baixo = np.percentile(df['gasto_por_hora'], 2.5)
percentil_alto = np.percentile(df['gasto_por_hora'], 97.5)

df_filtrado = df[(df['gasto_por_hora'] >= percentil_baixo) & (df['gasto_por_hora'] <= percentil_alto)]

df_total = df_filtrado.copy()

df_fev_maio = df_filtrado[(df_filtrado['date'].dt.month >= 2) & (df_filtrado['date'].dt.month <= 5)]

df_junho = df_filtrado[df_filtrado['date'].dt.month >= 6]


### 4. Agrupando os dataframes por clientes.
Criando as colunas:
 - media_gasto
 - max_gasto
 - min_gasto
 - desvio_padrao

In [114]:
df_treinamento = df_fev_maio.groupby(['clientCode', 'clientIndex']).agg(
    media_gasto=('gasto_por_hora', 'mean'),
    max_gasto=('gasto_por_hora', 'max'),
    min_gasto=('gasto_por_hora', 'min'),
    desvio_padrao=('gasto_por_hora', 'std')
).reset_index()

df_testes = df_junho.groupby(['clientCode', 'clientIndex']).agg(
    media_gasto=('gasto_por_hora', 'mean'),
    max_gasto=('gasto_por_hora', 'max'),
    min_gasto=('gasto_por_hora', 'min'),
    desvio_padrao=('gasto_por_hora', 'std')
).reset_index()

df_total = df_total.groupby(['clientCode', 'clientIndex']).agg(
    media_gasto=('gasto_por_hora', 'mean'),
    max_gasto=('gasto_por_hora', 'max'),
    min_gasto=('gasto_por_hora', 'min'),
    desvio_padrao=('gasto_por_hora', 'std')
).reset_index()

In [115]:
df_treinamento = df_treinamento.dropna()
df_testes = df_testes.dropna()
df_total = df_total.dropna()

#### 4.1. Visualização de como ficaram os novos dataframes

In [ ]:
df_treinamento #dataframe para treinamento do modelo

In [ ]:
df_testes #dataframe para testagem do modelo

#### 4.2. Normalização do banco de dados e retirada das colunas clienteCode e clienteIndex.

In [118]:
scaler = StandardScaler()
df_kmeans_treinamento = df_treinamento.copy()
df_kmeans_treinamento = df_kmeans_treinamento.drop(columns=['clientCode', 'clientIndex'])

df_scaled_treinamento = scaler.fit_transform(df_kmeans_treinamento)

### 5. Realiza a busca pelos melhores hiperparâmetros para o KMeans por meio do Random Search.
Ao achar os melhores hiperparâmetros, o kmeans é gerado para separar o código por clusters (grupos) considerando semelhança entre os mesmos.

In [ ]:
param_distributions = {'n_clusters': range(2, 10), 'init': ['k-means++', 'random'], 'n_init': [10, 20], 'max_iter': [300, 400]}
kmeans = KMeans(random_state=42)

random_search = RandomizedSearchCV(estimator=kmeans, param_distributions=param_distributions, n_iter=10, cv=5, random_state=42)
random_search.fit(df_scaled_treinamento)

best_kmeans = random_search.best_estimator_
print("Melhores parâmetros:", random_search.best_params_)

y_predict = best_kmeans.predict(df_scaled_treinamento)


Plota o gráfico dos clusters utilizando o PCA para reduzir a dimensionalidade.

In [ ]:
pca_2d = PCA(n_components=2)
df_pca_2d = pca_2d.fit_transform(df_scaled_treinamento)

fig = plt.figure(figsize=(14, 7))
ax2 = fig.add_subplot(122)

sns.scatterplot(x=df_pca_2d[:, 0], y=df_pca_2d[:, 1], hue=y_predict, s=100, alpha=0.75, 
                palette="Set1", edgecolor='white', linewidth=0.8, ax=ax2)

ax2.set_title(f"2D Clusters: {best_kmeans.n_clusters}")
ax2.set_xlabel("Componente Principal 1")
ax2.set_ylabel("Componente Principal 2")
ax2.grid(True)

plt.show()

#### 5.1. Plota os gráficos dos clusters para cada combinação de colunas possíveis. 
Isso é feito para ter uma melhor análise do comportamento dos clusters e identificar quais consideramos anômalos.

In [ ]:
df_treinamento['cluster'] = y_predict
df_kmeans_treinamento['cluster'] = y_predict

columns_to_plot = df_kmeans_treinamento.columns
column_combinations = list(combinations(columns_to_plot, 2))

for col_x, col_y in column_combinations:
    plt.figure(figsize=(10, 6))
    
    sns.scatterplot(x=df_kmeans_treinamento[col_x], y=df_kmeans_treinamento[col_y], hue=y_predict, 
                    palette="Set1", s=100, alpha=0.75, edgecolor='white', linewidth=0.8)

    plt.title(f"Scatter Plot - {col_x} vs {col_y}")
    plt.xlabel(col_x)
    plt.ylabel(col_y)
    plt.grid(True)
    plt.show()


Analisando os gráficos, é possível perceber que os clusters 5 e 6 apresentam um comportamento mais anômalo. Isso se deve ao fato de serem os clusters com maior desvio padrão, além de apresentarem um gasto mínimo muito baixo e um gasto máximo muito alto, evidenciando uma grande variação no consumo. Esse comportamento é considerado estranho neste caso.

Normalização do banco de dados e retirada das colunas clienteCode e clienteIndex.

In [122]:
df_kmeans_teste = df_testes.copy()
df_kmeans_teste = df_kmeans_teste.drop(columns=['clientCode', 'clientIndex'])

df_scaled_teste = scaler.transform(df_kmeans_teste)

### 6. Cálculo da distância de cada ponto do DataFrame de teste em relação aos centróides do KMeans gerado com o DataFrame de treinamento. 
Isso foi feito para adicionar a coluna de cluster ao DataFrame de teste, associando cada ponto ao centróide mais próximo.

In [123]:
closest_clusters, distances = pairwise_distances_argmin_min(df_scaled_teste, best_kmeans.cluster_centers_)

df_testes['cluster'] = closest_clusters

Visualizando como ficaram os novos dataframes

In [ ]:
df_testes #dataframe para testes considerando a coluna de cluster

In [ ]:
df_treinamento #dataframe para treinamento considerando a coluna de cluster

### 7. Definindo o modelo supervisionado

Retirada das colunas clientCode, clientIndex e cluster, para gerar o modelo supervisionado. A coluna cluster é justamente retirada pois como já identificamos e rotulamos os perfis anômalos, a coluna de clusters não será mais necessária.

In [ ]:
df_treino = df_treinamento.copy()
df_treino = df_treino.drop(columns=['clientCode', 'clientIndex'])

df_teste = df_testes.copy()
df_teste = df_testes.drop(columns=['clientCode', 'clientIndex'])

df_total_sem_id = df_total.copy()
df_total_sem_id = df_total.drop(columns=['clientCode', 'clientIndex'])

df_teste


Este código realiza a otimização de hiperparâmetros para quatro modelos de classificação (Random Forest, Gradient Boosting, Regressão Logística e SVC) utilizando o Random Search. 

In [ ]:
X_train = df_treino.drop(columns=['cluster'])
y_train = df_treino['cluster']

if 'cluster_pred' in df_teste.columns:
    df_teste = df_teste.drop(columns=['cluster_pred'])

X_test = df_teste.drop(columns=['cluster'])
y_test = df_teste['cluster'] if 'cluster' in df_teste.columns else None

models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000),
    'SVC': SVC(random_state=42, probability=True)
}

param_distributions = {
    'RandomForest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    'GradientBoosting': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    },
    'LogisticRegression': {
        'C': [0.1, 1, 10],
        'solver': ['liblinear', 'lbfgs']
    },
    'SVC': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    }
}

best_models = {}
for model_name, model in models.items():
    print(f"Tuning {model_name}...")

    random_search = RandomizedSearchCV(model, param_distributions[model_name], n_iter=10, cv=5, random_state=42)
    random_search.fit(X_train, y_train)
    
    best_model = random_search.best_estimator_
    best_models[model_name] = best_model
    
    y_pred = best_model.predict(X_test)
    
    df_testes['cluster_pred'] = y_pred
    
    print(f"\n{model_name} - Melhores parâmetros: {random_search.best_params_}")
    if y_test is not None:
        print(f"Acurácia: {accuracy_score(y_test, y_pred)}")
        print(f"Relatório de Classificação:\n{classification_report(y_test, y_pred)}")

        cm = confusion_matrix(y_test, y_pred)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['0','1', '2', '3', '4', '5', '6', '7'])
        disp.plot(cmap='Blues')
        plt.title(f'Matriz de Confusão - {model_name}')
        plt.show()
    else:
        print(f"Predições feitas, mas y_test não está disponível para calcular métricas.")

Após encontrar os melhores parâmetros, treina os modelos no conjunto de treino e faz previsões dos clusters no conjunto de teste, adicionando as previsões na coluna cluster_pred do DataFrame de teste. Por fim, avalia o desempenho do modelo calculando acurácia, exibindo o relatório de classificação e plotando a matriz de confusão para comparar predições com os rótulos reais.

Considerando as matrizes de confusão, temos que, a partir dos dados de teste (1 mês da base referida), o melhor modelo é o SVC - bastante eficaz para problemas de classificação, especialmente quando o número de características (features) é alto, e é usado em diversas aplicações, como detecção de anomalias.

### 8. Avaliação da importência das features

In [ ]:
result = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)

sorted_idx = result.importances_mean.argsort()
plt.barh(X_train.columns[sorted_idx], result.importances_mean[sorted_idx])
plt.xlabel("Importância")
plt.title("Importância das Features")
plt.show()

O gráfico de importância das features do modelo SVC destaca os seguintes pontos principais:

1. **max_gasto**: A variável mais influente, indicando que gastos elevados são fortes sinais de anomalias.

2. **desvio_padrao**: Também significativo, sugere que a variabilidade nos gastos é importante para identificar comportamentos atípicos.

3. **media_gasto**: Tem um impacto menor, mas ainda relevante, mostrando que os gastos habituais podem estar relacionados a anomalias.

4. **min_gasto**: Com a menor importância, indica que gastos mínimos não são determinantes para a detecção de anomalias.

Em suma, os gastos máximos e a variabilidade são os principais fatores na identificação de anomalias, enquanto a média e o mínimo gasto têm menor relevância.

In [ ]:
df_total_sem_id

In [ ]:
df_treino

In [131]:
svc_model = best_models.get('SVC')

cluster_pred = svc_model.predict(df_total_sem_id)

df_total['cluster'] = cluster_pred

### 9. Adição da coluna anomaly nos DataFrames
Como analisado no código anterior, os clusters 5 e 6 foram considerados anômalos. Portanto, os clientes que pertencem a esses clusters são classificados como anomalias, enquanto os demais são considerados normais.

In [ ]:
df_total['anomaly'] = ((df_total['cluster'] == 5) | (df_total['cluster'] == 6)).astype(int)

df_total.to_csv('../data/df_anomaly_cliente.csv')

df_total

Salvando o modelo

In [ ]:
joblib.dump(svc_model, 'svc_model.pkl')